In [1]:
import cv2
import numpy as np
import math
import os
from PIL import ImageGrab
import os.path as osp
from ultralytics import YOLO, RTDETR
import pydirectinput
import time
import copy
# load model
#model = YOLO("runs/detect/by_yolov8x_imgsize160/weights/best.pt")
#model = RTDETR("runs/detect/by_RTDETR_imgsize160\weights/best.pt")
model = YOLO("train9 - yolo5s.pt")

# 화면 크기 설정
screen_size = (1280, 720)
#screen_size = (1920, 1080)

# 시야각
fov = 103
angrat=math.tan(math.radians(fov / 2))
# 중앙 300x300 영역 설정
center_size=300
center_half=center_size/2
center_x = (screen_size[0] - center_size) // 2
center_y = (screen_size[1] - center_size) // 2
bbox = (center_x, center_y, center_x + center_size, center_y + center_size)
# 저장 경로 디폴트셋팅
train_path = "images/"
label_path = "labels/"

# 이미지 저장할 폴더 생성
if not osp.isdir(train_path):
    os.makedirs(train_path)

# 텍스트 파일 저장할 폴더 생성
if not osp.isdir(label_path):
    os.makedirs(label_path)


# 파일 열기 함수
def open_file(filename, mode):
    try:
        return open(filename, mode)
    except FileNotFoundError:
        return open(filename, "w")

# 폴더 내 파일 개수 확인
def file_count(path):
    global count
    folder_path = path
    file_list = os.listdir(folder_path)
    count = len(file_list)
    return count

# 박스의 w/2,h/2 는 size의 크기와 같도록 함
size=5

count = file_count(train_path)

# 클릭 좌표 관련 변수
pointlist=[]
# 캐릭터별 번호
hero_num=1
# 라벨링 모드
menu=""
# 상태 관련 bool
while_rec=False
charic_select=False

# 마우스 이벤트
def mouse_callback(event, x, y, flags, param):
    global pointlist,  charic_select
    global menu, size, hero_num

    # 좌클릭시 pointlist에 현재 캐릭터, 좌표 추가
    if event == cv2.EVENT_LBUTTONDOWN and while_stop==True:
        point = [x,y,size,hero_num]
        pointlist.append(point)
        if charic_select==True:
            charic_select=False
            menu=""
    
    elif event == cv2.EVENT_MOUSEWHEEL:
        # 캐릭터 종류 변경
        if charic_select==True:
            if flags > 0:
                hero_num += 1
            else:
                hero_num -= 1
        else:
            # 마우스 휠에 따라 사각형 크기 조절
            if flags > 0:
                size += 1
            elif size > 1:
                size -= 1
        if hero_num>39:
                hero_num=1
        elif hero_num<1:
            hero_num=39

heroes={0: 'bot', 1: 'ana', 2: 'ashe', 3: 'baptiste', 4: 'bastion', 5: 'brigitte', 6: 'cassidy', 7: 'd.va', 8: 'doomfist', 9: 'echo',
        10: 'genji', 11: 'hanzo', 12: 'illari', 13: 'junkerqeen', 14: 'junkrat', 15: 'kiriko', 16: 'lifeweaver', 17: 'lucio', 18: 'mauga', 19: 'mei',
        20: 'mercy', 21: 'moira', 22: 'orisa', 23: 'pharah', 24: 'ramattra', 25: 'reaper', 26: 'reinhardt', 27: 'roadhog', 28: 'sigma', 29: 'sojourn', 
        30: 'soldier:76',  31: 'sombra', 32: 'symmetra', 33: 'torbjorn', 34: 'tracer', 35: 'widowmaker', 36: 'winston', 37: 'wreckingball', 38: 'zarya', 39: 'zenyatta'}

녹화

In [3]:
# 녹화할 파일명과 코덱 설정
output_file = 'center_screen_recording_alone.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# 비디오 녹화 객체 생성
out = cv2.VideoWriter(output_file, fourcc, 20.0, (center_size, center_size))
while True:
    # 현재 화면 캡쳐
    img = ImageGrab.grab(bbox=bbox)
    # 화면에 표시 및 파일에 저장
    img=np.array(img)
    cv2.imshow('Center Screen Recording', img)
    out.write(img)
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) == ord('q'):
        break
# 사용한 자원 해제
out.release()
cv2.destroyAllWindows()

YOLO용 라벨링 함수(클래스, x, y, w, h)

In [54]:
# 마우스 클릭 좌표 기준 사각형 그리기
def stopped():
    global train_path, count
    global pointlist, while_stop, charic_select, menu
    while_stop=True

    while True:
        key=cv2.waitKey(10)
        tempimg=img.copy()

        # 상태바 & 박스 그리기
        cv2.putText(tempimg, f"{menu}stop count:{count} \n tag : {heroes[hero_num]}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
        if len(pointlist)>0:
            for point in pointlist:
                cv2.rectangle(tempimg, (point[0]-point[2], point[1]-point[2]), (point[0]+point[2], point[1]+point[2]), (255-point[3]*6, point[3]*6, -abs(6*(point[3]-20))+120))
        cv2.imshow("win",tempimg)

        if key == ord(" "):
            if  len(pointlist)>0:
                count = file_count(train_path)
                text = open_file(label_path + str(count) + ".txt", "w")
                cv2.imwrite(train_path + str(count) + '.jpg', img)
                for point in pointlist:
                    text.write(f"{point[3]} {float(point[0])/center_size} {float(point[1])/center_size} {float(point[2])*2/center_size} {float(point[2])*2/center_size}")
                    text.write("\n")
                text.close()
            break

        # 마지막 박스 지우기
        elif key == ord("x"):
            if len(pointlist)>0:
                pointlist.pop()
        # 캐릭터 변경 모드
        elif key == ord("z"):
            charic_select=True
            menu="charic change-"

    #변수 초기화
    while_stop=False
    pointlist=[]


simple모드 (x,y좌표만)

In [2]:
# 마우스 클릭 좌표 기준 사각형 그리기
def stopped():
    global train_path, count
    global pointlist, while_stop, charic_select, menu
    while_stop=True

    while True:
        key=cv2.waitKey(10)
        tempimg=img.copy()

        # 상태바 & 박스 그리기
        cv2.putText(tempimg, f"{menu}stop count:{count} \n tag : {heroes[hero_num]}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
        if len(pointlist)>0:
            for point in pointlist:
                cv2.rectangle(tempimg, (point[0]-point[2], point[1]-point[2]), (point[0]+point[2], point[1]+point[2]), (255-point[3]*6, point[3]*6, -abs(6*(point[3]-20))+120))
        cv2.imshow("win",tempimg)

        if key == ord(" "):
            if  len(pointlist)>0:
                count = file_count(train_path)
                text = open_file(label_path + str(count) + ".txt", "w")
                cv2.imwrite(train_path + str(count) + '.jpg', img)
                for point in pointlist:
                    text.write(f"{float(point[0])/center_size} {float(point[1])/center_size}")
                    text.write("\n")
                text.close()
            break

        # 마지막 박스 지우기
        elif key == ord("x"):
            if len(pointlist)>0:
                pointlist.pop()
        # 캐릭터 변경 모드
        elif key == ord("z"):
            charic_select=True
            menu="charic change-"

    #변수 초기화
    while_stop=False
    pointlist=[]

라벨링

D: 60프레임 이후로 스킵
A: 60프레임 이전으로 스킵
Space: 화면 정지
좌클릭: 중앙점 생성, 정사각형 생성
횔업&다운: 정사각형 크기 늘리고 줄이고
Z: 캐릭터 선택
X: 최근에 선택한 박스 취소

In [ ]:
readvideo=cv2.VideoCapture("center_screen_recording.avi")
cv2.namedWindow('win')
cv2.setMouseCallback("win", mouse_callback)
ret,img=readvideo.read()

while ret:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    key = cv2.waitKey(33)
    
    # 프레임 조정
    if key == ord("d"):
        readvideo.set(cv2.CAP_PROP_POS_FRAMES, readvideo.get(cv2.CAP_PROP_POS_FRAMES) + 60)
    elif key == ord("a"):
        readvideo.set(cv2.CAP_PROP_POS_FRAMES, readvideo.get(cv2.CAP_PROP_POS_FRAMES) - 60)

    # 사각형 생성
    elif key == ord(" "):
        stopped()

    # 영상 종료
    elif key == ord("q"):
        break
    
    ret,img=readvideo.read()

    cv2.imshow("win", img)

readvideo.release()
cv2.destroyAllWindows()

마우스 이동(YOLO)

In [ ]:
# 비디오 캡쳐
while True:
    img = ImageGrab.grab(bbox=bbox)
    results=model(img)
    for r in results:
        if len(r.boxes.xywh)>0:
                result = r.boxes.xywh[0].cpu().numpy()
                result = result.tolist()
                x, y, w, h= result
                #디폴트
                #pydirectinput.moveRel(round(x-150), round(y-150), relative=True)
                #제곱
                #pydirectinput.moveRel(round((x-150)*abs(x-150)/80), round((y-150)*abs(x-150)/80), relative=True)
                #아크탄젠트
                #pydirectinput.moveRel(round(math.atan(x-150)*abs(x-150)), round(math.atan(y-150)*abs(y-150)), relative=True)
                #시야각형식+속도 고려
                pydirectinput.moveRel(round((x-center_half)*max(abs(math.atan(x-center_half)),1) / angrat), round((y-center_half)*max(abs(math.atan(y-center_half)),1) / angrat), relative=True)

CNN 데이터셋 라벨링 확인용

In [ ]:
# 이미지 경로와 레이블 경로를 변수로 선언
pth = "./images"
lth = "./labels"

# 이미지 목록과 레이블 목록을 생성
img_list = sorted(os.listdir(pth))
lab_list = sorted(os.listdir(lth))

# 이미지와 레이블을 읽음
for i in range(len(img_list)):
    if i ==653:
        continue
    im = cv2.imread(pth + '/' + str(i) + ".jpg")
    with open(lth + '/' + str(i) + ".txt","r") as lbr:
        ll=lbr.read()
        ll=list(ll.split())
    ll=[round(center_size*float(x)) for x in ll]    
    cv2.circle(im, (ll[0],ll[1]), 5, (0,255,0), 2)
    # 텍스트 추가
    cv2.putText(im, str(i), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # 이미지 출력
    cv2.imshow("chek", im)
    cv2.waitKey(0)

# 모든 창 닫음
cv2.destroyAllWindows()

canny 필터 적용, canny_path에 변환된 이미지 저장

In [ ]:
canny_path="canny/"
if not osp.isdir(canny_path):
    os.makedirs(canny_path)
# 이미지 경로와 레이블 경로를 변수로 선언
pth = "./images"
lth = "./labels"

# 이미지 목록과 레이블 목록을 생성
img_list = sorted(os.listdir(pth))
lab_list = sorted(os.listdir(lth))

# 이미지와 레이블을 읽음
for i in img_list:
    num = i.find(".")
    im = cv2.imread(pth + '/' + i[:num] + ".jpg")
    im = cv2.resize(im, (center_size,center_size))
    im = cv2.Canny(im, 100, 200)
    cv2.imwrite(canny_path + i[:num] + ".jpg", im)
    with open(lth + '/' + i[:num] + ".txt","r") as lbr:
        ll=lbr.read()
        ll=list(ll.split())
    ll=[round(center_size*float(x)) for x in ll]    
    cv2.circle(im, (ll[0],ll[1]), 5, (255,255,255), 2)
    # 텍스트 추가
    cv2.putText(im, str(i), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # 이미지 출력
    cv2.imshow("chek", im)
    key=cv2.waitKey(1)
    if key==ord('q'):
        break

# 모든 창 닫음
cv2.destroyAllWindows()

cnn용으로 만든 라벨들을 yolo라벨로 변환

In [2]:
cv2.namedWindow('win')
cv2.setMouseCallback("win", mouse_callback)

changed_label_path="yololabel/"
if not osp.isdir(changed_label_path):
    os.makedirs(changed_label_path)

# 이미지 경로와 레이블 경로를 변수로 선언
pth = "./images"
lth = "./labels"

# 이미지 목록과 레이블 목록을 생성
img_list = os.listdir(pth)
lab_list = os.listdir(lth)
changed_lab_list = os.listdir(changed_label_path)

# 이미지와 레이블을 읽음
for i in img_list:

    key=""
    num=i.find(".")
    
    if i[:num]+".txt" not in changed_lab_list:
        while key!=ord(" "):
            img = cv2.imread(pth + '/' + i[:num] + ".jpg")
            img = cv2.resize(img, (center_size,center_size))
            img_temp=img
            #cv2.imwrite(canny_path + i[:num] + ".jpg", im)

            with open(lth + '/' + i[:num] + ".txt","r") as lbr:
                ll=lbr.read()
                ll=list(ll.split())
            ol=ll
            ll=[round(center_size*float(x)) for x in ll]

            cv2.rectangle(img_temp, (ll[0]-size, ll[1]-size), (ll[0]+size, ll[1]+size), (0, 0, 255) if charic_select==True else (255,255,255), 1)
            cv2.putText(img_temp, f" hero : {heroes[hero_num]}", (10, ll[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
            # 텍스트 추가
            cv2.putText(img_temp, str(i), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            
            # 이미지 출력
            cv2.imshow("win", img_temp)
            key=cv2.waitKey(1)

            if key == ord("q"):
                break
            elif key == ord("z"):
                if charic_select==True:
                    charic_select=False
                else:
                    charic_select=True
        with open(changed_label_path + '/' + i[:num] + ".txt","w") as lbr:
                lbr.write(f"{hero_num} {ol[0]} {ol[1]} {float(size*2)/center_size} {float(size*2)/center_size}")
    if key==ord('q'):
        break

# 모든 창 닫음
cv2.destroyAllWindows()

yolo라벨링 확인용

In [23]:
cv2.namedWindow('win')
cv2.setMouseCallback("win", mouse_callback)

# 이미지 경로와 레이블 경로를 변수로 선언
pth = "./imgNlabel/train/images"
lth = "./imgNlabel/train/labels"

# 이미지 목록과 레이블 목록을 생성
img_list = sorted(os.listdir(pth))
lab_list = sorted(os.listdir(lth))

# 이미지와 레이블을 읽음
for i in img_list:

    key=""
    num=i.find(".")

    img = cv2.imread(pth + '/' + i[:num] + ".jpg")
    img = cv2.resize(img, (center_size,center_size))
    with open(lth + '/' + i[:num] + ".txt","r") as lbr:
            ll=lbr.read()
            ll=list(ll.split())

    for j in range(len(ll)):
        if j==0:
            ll[j]=int(float(ll[j]))
        else:
            ll[j]=round(center_size*float(ll[j]))

    hero_num=ll[0]
    size=round(ll[3]//2)

    while key!=ord(" "):

        img_temp=img.copy()


        cv2.rectangle(img_temp, (ll[1]-size, ll[2]-size), (ll[1]+size, ll[2]+size), (0, 0, 255) if charic_select==True else (255,255,255), 1)
        cv2.putText(img_temp, f" hero : {heroes[hero_num]}", (10, ll[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
        # 텍스트 추가
        cv2.putText(img_temp, str(i), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        # 이미지 출력
        cv2.imshow("win", img_temp)
        key=cv2.waitKey(0)

        if key == ord("q"):
            break
        elif key == ord("a"):
            with open(lth + '/' + i[:num] + ".txt","w") as lbr:
                lbr.write(f"{hero_num} {float(ll[1])/center_size} {float(ll[2])/center_size} {float(size*2)/center_size} {float(size*2)/center_size}")
            break

        elif key ==ord("w"):
            size-=1
        elif key ==ord("e"):
            size+=1
        elif key ==ord("s"):
            hero_num-=1
        elif key ==ord("d"):
            hero_num+=1
        if size<0:
            size=1
        if hero_num>39:
            hero_num=0
        elif hero_num ==-1:
            hero_num=39
    
    if key==ord('q'):
        break

# 모든 창 닫음
cv2.destroyAllWindows()

파일 이름 변환 코드

In [15]:
# 이미지 경로와 레이블 경로를 변수로 선언
pth = "./imgNlabel/train/images"
lth = "./imgNlabel/train/labels"
img_list = sorted(os.listdir(pth))
lab_list = sorted(os.listdir(lth))

# 이미지 목록과 레이블 목록을 생성
img_list = sorted(os.listdir(pth))

max=len(img_list)
for i in range(max):
    os.rename(pth+ "/" + img_list[i], pth + "/img" +str(i+2898) + ".jpg")
for i in range(max):
    os.rename(lth+ "/" + lab_list[i], lth + "/img" +str(i+2898) + ".txt")

임의의 이미지와 라밸 100개를 train에서 test 혹은 valid로 이동시키는 코드

In [37]:
import shutil
import random

# 이미지 경로와 레이블 경로를 변수로 선언
pth = "./imgNlabel/train/images"
lth = "./imgNlabel/train/labels"

img_list = sorted(os.listdir(pth))
lab_list = sorted(os.listdir(lth))

npth = "C:/Users/Administator/Desktop/새 폴더 (2)/images"
nlth = "C:/Users/Administator/Desktop/새 폴더 (2)/labels"

# 이미지 저장할 폴더 생성
if not osp.isdir(npth):
    os.makedirs(npth)

# 텍스트 파일 저장할 폴더 생성
if not osp.isdir(nlth):
    os.makedirs(nlth)

max=len(img_list)

ran=random.sample(range(0, max), 100)
for i in ran:
    shutil.move(pth+ "/" + img_list[i], npth)
    shutil.move(lth+ "/" + lab_list[i], nlth)